In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import lit,col,expr
from delta.tables import DeltaTable

In [0]:
data_month_1 = [
  ('dgb01', 'luffy', 20, 1), 
  ('dgb02', 'zoro', 22, 1),
  ('dgb03', 'sanji', 22, 1)
        ]
schema = StructType([
  StructField('employee_id', StringType(), True),
  StructField('employee_name', StringType(), True),
  StructField('employee_age', IntegerType(), True),
  StructField('month_no', IntegerType(), True)
])
df_month_1 = spark.createDataFrame(data_month_1, schema)
df_month_1.show()

+-----------+-------------+------------+--------+
|employee_id|employee_name|employee_age|month_no|
+-----------+-------------+------------+--------+
|      dgb01|        luffy|          20|       1|
|      dgb02|         zoro|          22|       1|
|      dgb03|        sanji|          22|       1|
+-----------+-------------+------------+--------+



In [0]:
data_month_2 = [
  ('dgb01', 'Monkey D luffy', 21, 2), 
  ('dgb04', 'nami', 24, 2)
        ]
        
df_month_2 = spark.createDataFrame(data_month_2, schema)

In [0]:
data_month_3 = [
  ('dgb02', 'Ronovara zoro', 23, 3),
        ]
df_month_3 = spark.createDataFrame(data_month_3, schema)
df_month_3.show()


+-----------+-------------+------------+--------+
|employee_id|employee_name|employee_age|month_no|
+-----------+-------------+------------+--------+
|      dgb02|Ronovara zoro|          23|       3|
+-----------+-------------+------------+--------+



In [0]:
data_month_4 = [
  ('dgb01', 'Monkey D luffy', 21, 4), 
  ('dgb03', 'prince sanji', 23, 4),
  ('dgb05', 'ussopu', 19, 4),
        ]
df_month_4 = spark.createDataFrame(data_month_4, schema)
df_month_4.show()


+-----------+--------------+------------+--------+
|employee_id| employee_name|employee_age|month_no|
+-----------+--------------+------------+--------+
|      dgb01|Monkey D luffy|          21|       4|
|      dgb03|  prince sanji|          23|       4|
|      dgb05|        ussopu|          19|       4|
+-----------+--------------+------------+--------+



In [0]:
df_stage=df_month_1.select('employee_id','employee_name',lit("Nochange").alias("OldName"),'employee_age','month_no')

In [0]:
df_stage.display()

employee_id,employee_name,OldName,employee_age,month_no
dgb01,luffy,Nochange,20,1
dgb02,zoro,Nochange,22,1
dgb03,sanji,Nochange,22,1


In [0]:
path='dbfs:/user/hive/warehouse/employee_scd2'

In [0]:
df_stage.write.mode("overwrite").format("delta").option("path",path).saveAsTable('Scd2')

In [0]:
deltaTable = DeltaTable.forPath(spark, f"{path}")

In [0]:
%sql
select * from Scd2

employee_id,employee_name,OldName,employee_age,month_no
dgb01,Monkey D luffy,luffy,20,1
dgb03,sanji,Nochange,22,1
dgb02,zoro,Nochange,22,1
dgb04,nami,NoChange,24,2


In [0]:
deltaTable.alias('target').merge(
    source=df_month_2.alias('source'),
    condition='target.employee_id=source.employee_id',
).whenMatchedUpdate(
    condition='target.employee_name != source.employee_name',
    set=t
).whenNotMatchedInsert(values=d).execute()

In [0]:

deltaTable.alias('target').merge(source=df.alias('source'),condition=jc).whenMatchedUpdate(condition=mc,set=t).whenNotMatchedInsert(values=d).execute()

In [0]:
jc='target.employee_id=source.employee_id'
mc='target.employee_name != source.employee_name'

In [0]:
d={"employee_id": "source.employee_id","employee_name": "source.employee_name",'OldName': lit('NoChange'),"employee_age": "source.employee_age","month_no": "source.month_no"}

In [0]:
t={'OldName':'target.employee_name','employee_name':'source.employee_name'}

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/employee_scd2',True)

Out[27]: True

In [0]:
mergeAndSaveDf(df_month_2,path,jc,mc,t,d)

In [0]:
mergeAndSaveDf(df_month_1,path)

In [0]:
def mergeAndSaveDf(df,path,jc={},mc={},t={},d={}):
    if DeltaTable.isDeltaTable(spark,f"{path}"):
        deltaTable = DeltaTable.forPath(spark, f"{path}")
        deltaTable.alias('target').merge(source=df.alias('source'),condition=jc).whenMatchedUpdate(condition=mc,set=t).whenNotMatchedInsert(values=d).execute()
    else:
        df_stage=df.select('employee_id','employee_name',lit("Nochange").alias("OldName"),'employee_age','month_no')
        df_stage.write.mode("overwrite").format("delta").option("path",path).saveAsTable('Scd3')